# Lab 2
+ ## Автор: Роман Кривохижа
+ ## Група: ІС-72
+ ## Викладач: Новікова П.А.

****
****
****

## Module importing

In [620]:
import numpy as np
import pandas as pd

## Implement algorithm

In [49]:
class Rand:
    @staticmethod
    def exp(time_mean):
        return -time_mean * np.log(np.random.rand())

In [641]:
class Model:
    def __init__(self, delay0, delay1, maxQ=0, print_logs=False):
        '''
        :param delay0: mean delay of create
        :param delay1: mean delay of process
        :param maxQ: max size of queue
        :type delay0: float
        :type delay1: float
        :type maxQ: int
        '''
        self.delay_create = delay0
        self.delay_process = delay1
        # момент найближчої події
        self.tnext = 0.0
        # конкретний час
        self.tcurr = self.tnext
        # час завершення створення
        self.t0 = self.tcurr
        # час завершення обробки
        self.t1 = np.inf
        # max size of queue
        self.maxqueue = maxQ
        # number of created tasks
        self.num_create = 0
        # number of processed tasks
        self.num_process = 0
        # number of failure tasks
        self.failure = 0
        # current state
        self.state = 0
        # current size of queue
        self.queue = 0
        # ind of next event
        self.next_event = 0
        
        self.delta_t_history = []
        self.queue_history = []
        self.print_logs = print_logs
        
    def simulate(self, time_modeling):
        self.time_modeling = time_modeling
        while self.tcurr < self.time_modeling:
            self.tnext = self.t0
            self.next_event = 0
            
            # знайти найменший із моментів часу «момент надходження вимоги у систему»
            # та «момент звільнення пристрою обслуговування» і запам’ятати,
            # якій події  він відповідає
            if self.t1 < self.tnext:
                self.tnext = self.t1
                self.next_event = 1
            
            # просунути поточний час у момент найближчої події
            self.tcurr = self.tnext
            
            # виконати подію, яка відповідає моменту найближчої події
            if self.next_event == 0:
                self.event0()
            elif self.next_event == 1:
                self.event1()
            else:
                pass
            self.delta_t_history.append(self.tcurr)
            self.queue_history.append(self.queue)
            
            if self.print_logs:
                self.print_info()
        self.print_statistic()
        self.print_L_aver()
        self.print_Q_aver()
        self.print_p_fail()
        
    def print_statistic(self):
        print(f'num_create = {self.num_create}; num_process = {self.num_process}; failure = {self.failure}')
        
    def print_info(self):
        print(f't = {self.tcurr}; state = {self.state}; queue = {self.queue}')
        
    def print_L_aver(self):
        """
        Середня довжина черги черги
        """
        t = [self.delta_t_history[0]]
        for i in range(0, len(self.delta_t_history)-1):
             t.append(self.delta_t_history[i+1] - self.delta_t_history[i])
            
        self.l_aver = np.sum(np.array(t) * np.array(self.queue_history)) / self.time_modeling
        print(f'Середня довжина черги черги = {self.l_aver}')
        
    def print_Q_aver(self):
        """
        Середній час очікування в черзі
        """
        t = [self.delta_t_history[0]]
        for i in range(0, len(self.delta_t_history)-1):
             t.append(self.delta_t_history[i+1] - self.delta_t_history[i])
            
        self.q_aver = np.sum(np.array(t) * np.array(self.queue_history)) / self.num_process
        print(f'Середній час очікування в черзі = {self.q_aver}')
    
    def print_p_fail(self):
        self.p_fail = self.failure / self.num_create
        print(f'Ймовірність відмови: {self.p_fail}')
    
    def event0(self):
        # подія "натходження"
        self.t0 = self.tcurr + self.__get_delay_of_create()
        self.num_create += 1
        
        # якщо пристрій вільний, тоді встановлюємо стан "зайнятий"
        if self.state == 0:
            self.state = 1
            # запам'ятаємо час виходу
            self.t1 = self.tcurr + self.__get_delay_of_process()
        else:
            if self.queue < self.maxqueue:
                self.queue += 1
            else:
                self.failure += 1
                
    def event1(self):
        # подія "закінчилось обслуговування в пристрої"
        self.t1 = np.inf
        # стан "вільний"
        self.state = 0
        # якщо черга вимог не пуста, перемістити одну вимогу із черги у канал обслуговування
        if self.queue > 0:
            self.queue -= 1
            # стан "зайнятий"
            self.state = 1
            # запам'ятаємо час виходу
            self.t1 = self.tcurr + self.__get_delay_of_process()
        self.num_process += 1
        
    def __get_delay_of_create(self):
        return Rand.exp(self.delay_create)
    
    def __get_delay_of_process(self):
        return Rand.exp(self.delay_process)

In [653]:
model = Model(1, 3, 5, print_logs=True)
model.simulate(10)

t = 0.0; state = 1; queue = 0
t = 0.13967069283701933; state = 1; queue = 1
t = 0.2172082124635842; state = 1; queue = 2
t = 0.7128642717204219; state = 1; queue = 3
t = 1.350348713712247; state = 1; queue = 4
t = 3.2293835779503586; state = 1; queue = 3
t = 3.477968192399933; state = 1; queue = 2
t = 3.8300475753398953; state = 1; queue = 3
t = 4.3231380423354775; state = 1; queue = 2
t = 4.503110060673223; state = 1; queue = 3
t = 4.559659984267132; state = 1; queue = 4
t = 4.571190599531603; state = 1; queue = 5
t = 4.891739096424377; state = 1; queue = 5
t = 5.281471659055942; state = 1; queue = 5
t = 5.3429838916120245; state = 1; queue = 5
t = 5.345828807059387; state = 1; queue = 4
t = 5.566576603599525; state = 1; queue = 3
t = 6.333107664551815; state = 1; queue = 4
t = 7.135653708842899; state = 1; queue = 3
t = 7.807641507051389; state = 1; queue = 4
t = 8.474577157796286; state = 1; queue = 3
t = 9.816189432097746; state = 1; queue = 2
t = 10.386875244376036; state = 1; que

In [650]:
n_param = 10

delay0_list = list(range(1, n_param+1))
delay1_list = list(range(1, n_param+1))
maxQ_list = list(range(1, n_param+1))

time_modeling_list = [i*1000 for i in range(1, n_param+1)]

np.random.shuffle(delay0_list)
np.random.shuffle(delay1_list)
np.random.shuffle(maxQ_list)
np.random.shuffle(time_modeling_list)



df = pd.DataFrame()
rows = []

for i in range(n_param):
    print(f'{i+1} itteration')
    model = Model(delay0_list[i], delay1_list[i], maxQ_list[i], print_logs=False)
    model.simulate(time_modeling_list[i])
    rows.append({'delay_create': delay0_list[i], 'delay_process': delay1_list[i], 'maxqueue': maxQ_list[i], 'time_modeling': model.time_modeling, 
                 'num_create': model.num_create, 'num_process': model.num_process, 'failure': model.failure, 'last_queue_cnt': model.queue,
                 'l_aver': model.l_aver, 'q_aver': model.q_aver, 'p_fail': model.p_fail})
    print()
df = df.append(rows)

1 itteration
num_create = 2287; num_process = 2253; failure = 32
Середня довжина черги черги = 0.40306373136781964
Середній час очікування в черзі = 1.6101081146517429
Ймовірність відмови: 0.013992129427197202

2 itteration
num_create = 1468; num_process = 837; failure = 628
Середня довжина черги черги = 1.1728590267925119
Середній час очікування в черзі = 4.20379579495524
Ймовірність відмови: 0.42779291553133514

3 itteration
num_create = 1723; num_process = 1530; failure = 184
Середня довжина черги черги = 4.540809701872387
Середній час очікування в черзі = 29.678494783479653
Ймовірність відмови: 0.10679048171793383

4 itteration
num_create = 249; num_process = 181; failure = 63
Середня довжина черги черги = 3.2931761130348285
Середній час очікування в черзі = 36.388686331876556
Ймовірність відмови: 0.25301204819277107

5 itteration
num_create = 6107; num_process = 669; failure = 5436
Середня довжина черги черги = 0.8907377762342188
Середній час очікування в черзі = 7.988679607481782

In [651]:
df

,delay_create,delay_process,maxqueue,time_modeling,num_create,num_process,failure,last_queue_cnt,l_aver,q_aver,p_fail
0,4,2,4,9000,2287,2253,32,1,0.403064,1.610108,0.013992
1,2,3,2,3000,1468,837,628,2,1.172859,4.203796,0.427793
2,6,6,9,10000,1723,1530,184,8,4.540810,29.678495,0.106790
3,9,10,5,2000,249,181,63,4,3.293176,36.388686,0.253012
4,1,9,1,6000,6107,669,5436,1,0.890738,7.988680,0.890126
5,10,1,6,5000,473,472,0,0,0.006075,0.064355,0.000000
6,7,5,3,4000,591,523,65,2,0.752107,5.752252,0.109983
7,5,7,10,1000,206,136,63,6,6.929987,50.955789,0.305825
8,3,4,8,8000,2655,2028,627,0,5.317261,20.975389,0.236158
9,8,8,7,7000,867,765,95,6,3.140882,28.740093,0.109573


На лабораторній роботі ми реалізовували алгоритм імітації простої моделі обслуговування способом, який орієнтований на події. Вихідні результати моделі показали, що зміна вхідних параметрів суттєво впливає на кінцеві результати. Основний недолік розробки імітації: якщо більше двох подій відбуваються в один момент часу, потрібно забезпечити, щоб вони відбувались у правильній послідовності.